<a href="https://colab.research.google.com/github/h-marshall69/NilvelLagoPuno/blob/main/RecopilacionDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montaje de Google Drive
from google.colab import drive
import os

# Montaje de Google Drive
drive.mount('/content/drive')

# Configuración de la carpeta de guardado
path_dir = '/content/drive/MyDrive/LagoPuno'
os.makedirs(path_dir, exist_ok=True)
os.chdir(path_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import concurrent.futures
import time
import os

url = "https://www.senamhi.gob.pe/include/ajax-informacion-diaria.php"
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0',
}

def filtrar_datos(data, estaciones):
    return [entry for entry in data['content'] if entry['nomEsta'] in estaciones]

def obtener_datos(fecha, hora, estaciones):
    fecha_str = fecha.strftime('%Y-%m-%d')
    data = {
        'fecha': fecha_str,
        'hora': hora
    }

    try:
        response = requests.post(url, headers=headers, data=data)
        response.raise_for_status()
        datos_filtrados = filtrar_datos(response.json(), estaciones)
        return fecha_str, datos_filtrados
    except requests.exceptions.RequestException as e:
        print(f"Error en la solicitud para {fecha_str}: {e}")
        return fecha_str, None

def reintentar_fechas_fallidas(fechas_fallidas, hora, estaciones, datos_recolectados):
    print(f"Reintentando {len(fechas_fallidas)} fechas fallidas...")
    for fecha in fechas_fallidas:
        fecha_str, resultado = obtener_datos(fecha, hora, estaciones)
        if resultado:
            datos_recolectados.extend([{
                'fecha': fecha_str,
                'codZonal': dato['codZonal'],
                'codEsta': dato['codEsta'],
                'nomEsta': dato['nomEsta'],
                'uniHidrografica': dato['uniHidrografica'],
                'nomDepa': dato['nomDepa'],
                'nomCuenca': dato['nomCuenca'],
                'nomSector': dato['nomSector'],
                'dato': dato['dato'],
                'unidad': dato['unidad'],
                'datAnomalia': dato['datAnomalia'],
                'uniAnomalia': dato['uniAnomalia'],
                'tendencia': dato['tendencia'],
                'umbralRojo': dato['umbralRojo'],
                'cuerpoAgua': dato['cuerpoAgua']
            } for dato in resultado])

def main():
    estaciones = ['MUELLE ENAFER']  # Añade aquí las estaciones que quieras monitorear
    fecha_inicio = datetime(2023, 10, 13)
    fecha_fin = datetime(2024, 10, 13)
    hora = '18:00'

    fechas = [fecha_inicio + timedelta(days=i) for i in range((fecha_fin - fecha_inicio).days + 1)]
    datos_recolectados = []
    fechas_fallidas = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futuro_a_fecha = {executor.submit(obtener_datos, fecha, hora, estaciones): fecha for fecha in fechas}
        for futuro in concurrent.futures.as_completed(futuro_a_fecha):
            fecha = futuro_a_fecha[futuro]
            fecha_str, resultado = futuro.result()
            if resultado:
                datos_recolectados.extend([{
                    'fecha': fecha_str,
                    'codZonal': dato['codZonal'],
                    'codEsta': dato['codEsta'],
                    'nomEsta': dato['nomEsta'],
                    'uniHidrografica': dato['uniHidrografica'],
                    'nomDepa': dato['nomDepa'],
                    'nomCuenca': dato['nomCuenca'],
                    'nomSector': dato['nomSector'],
                    'dato': dato['dato'],
                    'unidad': dato['unidad'],
                    'datAnomalia': dato['datAnomalia'],
                    'uniAnomalia': dato['uniAnomalia'],
                    'tendencia': dato['tendencia'],
                    'umbralRojo': dato['umbralRojo'],
                    'cuerpoAgua': dato['cuerpoAgua']
                } for dato in resultado])
            else:
                fechas_fallidas.append(fecha)  # Almacenar fechas con errores

            porcentaje_recolectados = (len(datos_recolectados) / (len(fechas) * len(estaciones))) * 100
            print(f"Datos recolectados: {len(datos_recolectados)} de {len(fechas) * len(estaciones)} posibles ({porcentaje_recolectados:.2f}%).")

    # Reintentar las fechas que dieron error
    reintentar_fechas_fallidas(fechas_fallidas, hora, estaciones, datos_recolectados)

    # Crear DataFrame y ordenar por fecha
    df = pd.DataFrame(datos_recolectados)
    df['fecha'] = pd.to_datetime(df['fecha'])  # Convertir 'fecha' a datetime
    df = df.sort_values('fecha')  # Ordenar por fecha

    csv_filename = os.path.join(path_dir, 'datos_estaciones_2024.csv')
    df.to_csv(csv_filename, index=False)
    print(f"Datos guardados en '{csv_filename}'.")

if __name__ == "__main__":
    main()

Datos recolectados: 25 de 367 posibles (6.81%).
Datos recolectados: 26 de 367 posibles (7.08%).
Datos recolectados: 27 de 367 posibles (7.36%).
Datos recolectados: 28 de 367 posibles (7.63%).
Datos recolectados: 29 de 367 posibles (7.90%).
Datos recolectados: 30 de 367 posibles (8.17%).
Datos recolectados: 31 de 367 posibles (8.45%).
Datos recolectados: 32 de 367 posibles (8.72%).
Datos recolectados: 33 de 367 posibles (8.99%).
Error en la solicitud para 2023-11-19: Expecting value: line 1 column 1 (char 0)
Datos recolectados: 33 de 367 posibles (8.99%).
Datos recolectados: 34 de 367 posibles (9.26%).
Datos recolectados: 35 de 367 posibles (9.54%).
Datos recolectados: 36 de 367 posibles (9.81%).
Datos recolectados: 37 de 367 posibles (10.08%).
Datos recolectados: 38 de 367 posibles (10.35%).
Datos recolectados: 39 de 367 posibles (10.63%).
Datos recolectados: 40 de 367 posibles (10.90%).
Datos recolectados: 41 de 367 posibles (11.17%).
Datos recolectados: 42 de 367 posibles (11.44%).
D

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import concurrent.futures
import logging
import time
import json
import os

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

url = "https://www.senamhi.gob.pe/include/ajax-informacion-mensual.php"
headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0',
}

def filtrar_datos(data, estaciones):
    try:
        return [entry for entry in data['content'][0]['detalle'] if entry['nomEsta'] in estaciones]
    except (KeyError, IndexError, TypeError) as e:
        logging.error(f"Error al filtrar datos: {e}")
        return []

def obtener_datos_mensuales(fecha, estaciones, max_retries=3):
    data = {
        'fecha': fecha.strftime('%Y-%m'),
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(url, headers=headers, data=data, timeout=10)
            response.raise_for_status()
            json_data = response.json()
            datos_filtrados = filtrar_datos(json_data, estaciones)
            return fecha.strftime('%Y-%m'), datos_filtrados
        except requests.exceptions.RequestException as e:
            logging.warning(f"Intento {attempt + 1} fallido para {fecha.strftime('%Y-%m')}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Espera exponencial entre intentos
            else:
                logging.error(f"Error en la solicitud para {fecha.strftime('%Y-%m')} después de {max_retries} intentos: {e}")
        except json.JSONDecodeError as e:
            logging.error(f"Error al decodificar JSON para {fecha.strftime('%Y-%m')}: {e}")
            break  # Si hay un error de JSON, no tiene sentido reintentar

    return fecha.strftime('%Y-%m'), None

def reintentar_fechas_fallidas(fechas_fallidas, estaciones, datos_recolectados):
    logging.info(f"Reintentando {len(fechas_fallidas)} fechas fallidas...")
    for fecha in fechas_fallidas:
        fecha_str, resultado = obtener_datos_mensuales(fecha, estaciones)
        if resultado:
            datos_recolectados.extend([{
                'fecha': fecha_str,
                'codZonal': dato.get('codZonal', ''),
                'codEsta': dato.get('codEsta', ''),
                'nomEsta': dato.get('nomEsta', ''),
                'uniHidrografica': dato.get('uniHidrografica', ''),
                'nomDepa': dato.get('nomDepa', ''),
                'nomSector': dato.get('nomSector', ''),
                'dato': dato.get('dato', ''),
                'datoAnt': dato.get('datoAnt', ''),
                'unidad': dato.get('unidad', ''),
                'datAnomalia': dato.get('datAnomalia', ''),
                'uniAnomalia': dato.get('uniAnomalia', ''),
                'tendencia': dato.get('tendencia', ''),
                'umbralRojo': dato.get('umbralRojo', ''),
                'cuerpoAgua': dato.get('cuerpoAgua', '')
            } for dato in resultado])

def main():
    estaciones = ['MUELLE ENAFER', 'OTRA ESTACION']
    fecha_inicio = datetime(1982, 10, 1)
    fecha_fin = datetime(2024, 9, 1)

    datos_recolectados = []
    fechas_fallidas = []
    fechas = []
    fecha_actual = fecha_inicio

    # Generar las fechas mensuales
    while fecha_actual <= fecha_fin:
        fechas.append(fecha_actual)
        fecha_actual += timedelta(days=32)
        fecha_actual = fecha_actual.replace(day=1)

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futuro_a_fecha = {executor.submit(obtener_datos_mensuales, fecha, estaciones): fecha for fecha in fechas}
        for futuro in concurrent.futures.as_completed(futuro_a_fecha):
            fecha = futuro_a_fecha[futuro]
            try:
                fecha_str, resultado = futuro.result()
                if resultado:
                    datos_recolectados.extend([{
                        'fecha': fecha_str,
                        'codZonal': dato.get('codZonal', ''),
                        'codEsta': dato.get('codEsta', ''),
                        'nomEsta': dato.get('nomEsta', ''),
                        'uniHidrografica': dato.get('uniHidrografica', ''),
                        'nomDepa': dato.get('nomDepa', ''),
                        'nomSector': dato.get('nomSector', ''),
                        'dato': dato.get('dato', ''),
                        'datoAnt': dato.get('datoAnt', ''),
                        'unidad': dato.get('unidad', ''),
                        'datAnomalia': dato.get('datAnomalia', ''),
                        'uniAnomalia': dato.get('uniAnomalia', ''),
                        'tendencia': dato.get('tendencia', ''),
                        'umbralRojo': dato.get('umbralRojo', ''),
                        'cuerpoAgua': dato.get('cuerpoAgua', '')
                    } for dato in resultado])
                else:
                    fechas_fallidas.append(fecha)  # Almacenar las fechas que fallaron
            except Exception as e:
                logging.error(f"Error al procesar los resultados para {fecha}: {e}")

    # Reintentar las fechas fallidas
    if fechas_fallidas:
        reintentar_fechas_fallidas(fechas_fallidas, estaciones, datos_recolectados)

    if datos_recolectados:
        # Crear DataFrame y ordenar por fecha
        df = pd.DataFrame(datos_recolectados)
        df['fecha'] = pd.to_datetime(df['fecha'])  # Convertir 'fecha' a datetime
        df = df.sort_values('fecha')  # Ordenar por fecha

        # Guardar en CSV
        csv_filename = os.path.join(path_dir, 'datos_mensuales_estaciones_2023-2024.csv')
        df.to_csv(csv_filename, index=False)
        logging.info(f"Datos guardados en '{csv_filename}'.")
    else:
        logging.warning("No se recolectaron datos. No se creará el archivo CSV.")

if __name__ == "__main__":
    main()
